In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website---------------------------------------------------------------------------
URL = "https://www.redbus.in/online-booking/north-bengal-state-transport-corporation"

# Initialize Driver----------------------------------------------------------------------------
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver


def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes----------------------------------------------------------------
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details---------------------------------------------------------------
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" Button
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details---------------------------------------------------------------------
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages-------------------------------------------------------------
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame------------------------------------------------------
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('north_bengal_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/siliguri-to-darjeeling: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF63766FDA5+29557]
	(No symbol) [0x00007FF6375E2240]
	(No symbol) [0x00007FF63749B6EA]
	(No symbol) [0x00007FF6374EFA15]
	(No symbol) [0x00007FF6374EFC6C]
	(No symbol) [0x00007FF63753BB07]
	(No symbol) [0x00007FF63751753F]
	(No symbol) [0x00007FF6375388A3]
	(No symbol) [0x00007FF6375172A3]
	(No symbol) [0x00007FF6374E12DF]
	(No symbol) [0x00007FF6374E2451]
	GetHandleVerifier [0x00007FF63799DCBD+3363469]
	GetHandleVerifier [0x00007FF6379E9B47+3674391]
	GetHandleVerifier [0x00007FF6379DEAEB+3629243]
	GetHandleVerifier [0x00007FF63772FC66+815670]
	(No symbol) [0x00007FF6375ED6EF]
	(No symbol) [0x00007FF6375E92B4]
	(No symbol) [0x00007FF6375E9450]
	(No symbol) [0x00007FF6375D81FF]
	BaseThreadInitThunk [0x00007FFBC57C257D+29]
	RtlUserThreadStart [0x00007FFBC702AF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus

NameError: name 'driver' is not defined

In [7]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,NBSTC-SLG-KOL-R1-18:30 - 156410,NON A/C Seater Push Back (2+2),18:30,12h 00m,06:30,3.9,INR 486,19 Seats available
1,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Poulami Paribahan,Volvo Multi-Axle A/C seater/sleeper (2+1),19:15,12h 15m,07:30,4.1,INR 1149,33 Seats available
2,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Express Line,Volvo 9600 A/C Seater/Sleeper (2+1),19:30,12h 30m,08:00,4.4,1199,46 Seats available
3,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,AdRoyal Cruiser,Volvo A/C Seater (2+2),17:35,13h 00m,06:35,3.8,995,16 Seats available
4,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,Shyamoli Paribahan Pvt Ltd,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:00,12h 05m,06:05,4.5,INR 1500,23 Seats available
...,...,...,...,...,...,...,...,...,...,...
344,Siliguri to Raiganj,https://www.redbus.in/bus-tickets/siliguri-to-...,Shree Krishna Travels,NON A/C Seater / Sleeper (2+2),20:00,06h 00m,02:00,3.4,INR 250,18 Seats available
345,Siliguri to Raiganj,https://www.redbus.in/bus-tickets/siliguri-to-...,Aradhana Bus Service,A/C Seater / Sleeper (2+2),21:00,04h 30m,01:30,3.5,760,6 Seats available
346,Siliguri to Raiganj,https://www.redbus.in/bus-tickets/siliguri-to-...,Aradhana Bus Service,NON A/C Seater / Sleeper (2+2),14:30,04h 45m,19:15,2.4,INR 700,33 Seats available
347,Siliguri to Raiganj,https://www.redbus.in/bus-tickets/siliguri-to-...,Lokenath (Maa Tara),NON A/C Seater / Sleeper (2+2),15:15,05h 00m,20:15,2.9,INR 280,11 Seats available
